# Spike-Triggered Average Analysis

This IPython notebook is written by [Sahar Pirmoradian](https://www.linkedin.com/in/spirmorad/) as part of [RENVISION](http://www.renvision-fp7.eu) project for computing spike-triggered average of cells that are recorded using a high-density multielectrode arrays. 

Version 2, 14 May 2015

Changes: 

(1) In order to analyse P79 data (shifted white noise) with different temporal and spatial resolutions compared to other white noise data we used to work with, I had to slightly modify the script: using int and float types more accurately. 

(2) Because of too many unexpected pauses in the stimulus presentation, I had to measure a more accurate estimation of the duration between stimulus frames. 

(3) In order to read hdf5 files with IIT format, read_hdf5() was modified. 

## Importing necessary libraries

In [154]:
import h5py
import numpy as np
import scipy.io
import os.path
from scipy import stats
import matplotlib as mplt
import matplotlib.pyplot as plt
import pylab
import scipy.optimize as opt

## Reading and preprocessing data

### Setting data and stimulus file names

In [155]:
masterdr = '/Users/sahar/Documents/spktanlz/data/retina/P79_28Oct14'

# data file is an hdf5 file (spikes, channels, ...)
datafullfn = masterdr + '/hdf5/P79_28_10_14_ret1_left_stim4_shifted4-white-noise_SpikesSortedRearrangeFinal_sorted.hdf5'

# stimulus file is a mat file that contains SamplingFrequency, frameSamples, frameTimes, pixels_allframes (see read_stimfile())  
stimfullfn = masterdr + '/mat/ret1_LE_shiftedWN_stim.mat'

#P38 params
#masterdr = '/Users/sahar/Documents/spktanlz/data/retina/P38_06Mar14/ret2'
#datafullfn = masterdr + '/hdf5/P38_06_03_14_retina02_lightstim_sorted.hdf5'
#stimfullfn = masterdr + '/mat/Retina02_LeftEye_Stim01_whitenoise100ms_stim.mat'

### Setting parameters 

In [156]:
figpath = masterdr + '/figs/' # where STA figures are stored
staparams_matfile = figpath + 'staparams' # it will be stored as a mat file
spksrc = 'su'  # spike source of our data, 'ch' (channels), 'cl' (clusters), 'su' (sorted units)
coord_spksrc = 'rc' # coordination of channles on the chip, 'rc' (row-column) or 'cr' (column-row) 
minamp = 0 # minimum amplitude of spikes
minspk = 30 # minimum spikes a cell must have in Full-field experiment to be considered in analyses; if the number of FF trials are 30, we assume a responding cell fires at least once to each trial, that is minimum spikes are 30
dur_sta = .5 # in sec, duration of STA 
dt_sta = 0.01 # in sec, temporal resolution of STA
dt_sta_graphics = .1 # in sec, graphically we show this temporal resolution on STA plots
spklatency_sta = .0  # always kept 0
figformat = '.png' # format of figures
max_var_acceptable = 20 # a fitted receptive field is acceptable if its center is fit with a variance that does not exceed this number
max_pval_acceptable = 1e-06 # p-value of STA peaks must not exceed this number
checkSTAPeakZFLG = True #if True, the STA peak is checked for significance
staplotFLG = True  #if True, STA plots are saved
PrintGaussParamFLG = False # if True, you can see the parameters of Gaussian fit
slctdchs_sta = np.array([275, 1071])# 'all' # if you like to compute STA of all cells, set slctdchs_sta = 'all'; 
                                    # if you compute only the STA of selected cells, e.g. 13 or 752, set slctdchs_sta = np.array([13,752])
                                    # if you compute the STA of a range of cells, e.g. from 10 to 20, set slctdchs_sta = np.arange(10,20) 
plotJustRespUnitsFLG = True # if True, you only save the STA plots of cells with a significant STA peak
staSaveParamsFLG = True
no_gausfit_params = 7
chip_size = 2.67 # in mm
tableau = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229), 
              (0,0,128)] # color
#converting to rgb
tableau = [(c[0]/255., c[1]/255., c[2]/255.) for c in tableau]
    
# TODO (not used in this version)
figpath_rate = '.' # where Firing Rate figures are stored
figpath_mosaic = '.' # where figures of spatial arrangements of cells are stored
calcBiasidx4staFLG = False 
mosaicplotFLG=False
args_fr = None
#changes 
mosaic_imgsize= 64*64 #729 #image size (27*27)
no_neighbpix = mosaic_imgsize  
#changes end
plotJustFFRespUnitsFLG=False

### Reading data file

In [157]:
def read_hdf5(fullfn, spksrc='ch'):
  # fullfn - string; full file name of an hdf5 file to be read
  # spksrc - 'ch' (channels), 'cl' (clusters), or 'su' (sorted units); spike source
  # OUTPUT:
  # spikes - array; frame number at which spikes were generated
  # chs    - array; channels or clusters that generated corresponding spikes 
  # amps   - array; the amplitude of spikes
  # freq   - float; sampling frequency. The frequency at which neuron signals were recorded
  # nbins  - int; the resolution of channel locations; it's usually 64.
  # loc2d  - array2D; the 2d location of channels on a chip   
  # loc    - array1D; the location of channels that has been converted to 1 dimension for simplicity
  # locspks2d - array2D; the location of spikes (not channels)
  try:  
    myfile = h5py.File(fullfn,'r')
  except:
    raise IOError('\nI cannot open your hdf5 file: %s' % fullfn)

  try:  
    spikes = np.array(myfile['Times'].value+spkframe_offset, dtype=int)
    amps = myfile['Amplitudes'].value
    freq = myfile['Sampling'].value
    loc2d, locspks2d = (np.array([]), np.array([]))
    if (spksrc == 'ch'):
      ch = myfile['Channels'].value
      loc = np.array([])
      nbins = 64
    elif (spksrc == 'cl'):
      locspks2d = myfile['Locations'].value
      ch = myfile['Cluster']['ClusterId'].value
      loc2d = myfile['Cluster']['CLocations'].value    
      loc = myfile['Cluster']['CLocationsS'].value  
      nbins = myfile['Cluster']['nBins'].value #768 #
      # removing clusters with id equal to -1
  #    pdb.set_trace()
      valididxs = np.where(ch>=0)
      spikes = spikes[valididxs]
      amps = amps[valididxs]
      ch = ch[valididxs]    
      locspks2d = locspks2d[valididxs]    
         
    elif (spksrc == 'su'):
      ch = myfile['SortedUnits'].value
      markedch = myfile['MarkedChannels'].value
      valididxs = np.where(markedch>-1)
      spikes = spikes[valididxs]
      amps = amps[valididxs]
      ch = ch[valididxs]    
      loc = myfile['LocationsS'].value
      nbins = 64  
  except:
    spikes = myfile['TimeStampMatrix']['timeStamp']
    ch = myfile['TimeStampMatrix']['idUnitStim']
    freq = myfile['GlobalParameter']['RecordingParameter']['samplingFreq'][0]
    m = myfile['NeuronMatrix']['marked']
    validchs_id = myfile['NeuronMatrix']['id'][np.where(m<1)]
    validchs_idx = np.where(np.in1d(ch, validchs_id))
    spikes = spikes[validchs_idx]
    ch = ch[validchs_idx]    
    amps = np.array([10] * len(spikes))
    nbins = 64  
    loc = (myfile['NeuronMatrix']['posX']-1)*nbins+(myfile['NeuronMatrix']['posY']-1)
    # to account for python arrays starting with 0, whereas neuron ids start with 1
    loc = np.insert(loc, 0, 0)
    loc2d, locspks2d = (np.array([]), np.array([]))    
  data = {'spikes':spikes, 'amps': amps, 'ch':ch, 'freq':freq, 'nbins': nbins, 'loc':loc, 'loc2d':loc2d, 'locspks2d': locspks2d}
  return data


In [158]:
data = read_hdf5(datafullfn, spksrc=spksrc)
data

{'amps': array([10, 10, 10, ..., 10, 10, 10]),
 'ch': array([2329, 1549, 1327, ..., 3385, 1216, 3836], dtype=int32),
 'freq': 7062,
 'loc': array([   0,    2,    3, ..., 1546, 1546, 1546], dtype=int32),
 'loc2d': array([], dtype=float64),
 'locspks2d': array([], dtype=float64),
 'nbins': 64,
 'spikes': array([     23,      27,      31, ..., 8225800, 8225800, 8225800], dtype=int32)}

### Reading stimulus file

In [159]:
def read_stimfile(fullfn):
  # fullfn - string; a mat file containing the stimulus file whose variables are:
  # SamplingFrequency - float; sampling frequency of data acquisition, e.g. 7022
  # frameTimes - horizontal vector; times at which stimuli frames were presented, e.g. in a 15min white noise presentation with 100msec duration, 9000 images are presented, thus frameTimes is a <1x9000 double> vector  
  # frameSamples - horizontal vector; instead of time, it contains frame numbers at which stimuli were presented, i.e. frameSamples = frameTimes * SamplingFrequency 
  # pixels_allframes - 2D array; each row represents the pixels of the image presented at each frame, e.g. in a 15min white noise presentation with 100msec duration, 9000 images are presented, and if each image has 27*27 (=729) pixels, then pixels_allframes is a <9000x729 double> array  
  myfile = scipy.io.loadmat(fullfn)
  stimframes = np.array(myfile.get('frameSamples')[0], dtype='int')
  stimtimes = np.array(myfile.get('frameTimes')[0])
  stimpixels = np.array(myfile.get('pixels_allframes'), dtype='int')
  stimfreq = np.array(myfile.get('SamplingFrequency')[0][0])
  stim = {'stimframes':stimframes, 'stimtimes': stimtimes, 'stimfreq':stimfreq, 'stimpixels':stimpixels}  
  return stim

In [160]:
stim = read_stimfile(stimfullfn) 
stim

{'stimframes': array([ 227407,  227642,  227877, ..., 8025584, 8025820, 8026055]),
 'stimfreq': array(7062.058198545425),
 'stimpixels': array([[0, 1, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        [1, 1, 1, ..., 1, 1, 1],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 1, 1, 1]]),
 'stimtimes': array([   32.20123562,    32.23451204,    32.26778845, ...,  1136.43696701,
         1136.47038503,  1136.50366145])}

## Defining handy functions that will be called in later steps

get_spkidx(data, minamp, minframe, maxframe) returns indices of spikes, with minimum amplitude of minamp, which are fired between minframe and maxframe

In [161]:
def get_spkidx(data, minamp=7, minframe=0, maxframe=1000):
  amps = data['amps']
  spikes = data['spikes']
  idxframe_min = np.where(spikes >= minframe)[0]
  elmidx_beforemaxf = next(x[0] for x in enumerate(idxframe_min) if spikes[x[1]] > maxframe)
  idxframe = idxframe_min[:elmidx_beforemaxf]
  idxamp = np.where(amps >= minamp)[0]
  idx = np.intersect1d(idxframe, idxamp)
  return idx

get_hist_spkcnt(data, spkidx) returns the histogram of spikes whose indices are spkidx

In [162]:
def get_hist_spkcnt(data, spkidx):
  ch = data['ch']
  no_ch=max(ch[spkidx]) + 1
  hist_spkcnt = np.histogram(ch[spkidx],bins=np.arange(no_ch+1))[0]
  return hist_spkcnt

get_chlocation(ich, spksrc, nbins, loc, coord_spksrc) returns the location (locr, locc) of channel ich

In [163]:
def get_chlocation(ich, spksrc, nbins, loc, coord_spksrc='rc'):
  if 'array' in type(loc[ich]).__name__:
    locr = loc[ich][0]
    locc = loc[ich][1]
  elif spksrc == 'ch':
    locr = ich / nbins
    locc = np.mod(ich, nbins)
  elif spksrc == 'cl' or spksrc == 'su':
    if nbins:
      locr = loc[ich] / nbins
      locc = np.mod(loc[ich], nbins)
      locr = locr * 64 / nbins
      locc = locc * 64 / nbins  
    else:
      locr = loc[ich][0]
      locc = loc[ich][1]
  if coord_spksrc == 'cr':
    tmp = locr
    locr = locc
    locc = tmp
  return locr, locc

get_neighborpixels(locr, locc, image_xsize, no_neighbpix_x) returns all pixels surrounding location [locr, locc]

In [164]:
def get_neighborpixels(locr, locc, image_xsize, no_neighbpix_x):
  # change in v1.2  
  minr = locr - int(no_neighbpix_x/2)
  maxr = locr + int(np.ceil(no_neighbpix_x/2)) 
  minc = locc - int(no_neighbpix_x/2)
  maxc = locc + int(np.ceil(no_neighbpix_x/2))
  # change end
  if minr < 0:
    maxr -= minr
    minr = 0
  if maxr > image_xsize:
    minr = minr + (image_xsize - maxr)
    maxr = image_xsize
  if minc < 0:
    maxc -= minc
    minc = 0
  if maxc > image_xsize:
    minc = minc + (image_xsize - maxc)
    maxc = image_xsize  
  neighborpixels_pos = [r*image_xsize+c for r in np.arange(minr, maxr) for c in np.arange(minc, maxc)]
  return neighborpixels_pos, minr, minc

plotsta(ich, \*args, \**kwds) plots temporal and spatial course of STA of channel ich

In [165]:
def plotsta(ich, sta_ich, neighborpixels_pos, spkcnt_ich, dt_sta_graphics, time_bins,figpath,spksrc,\
              figformat,maxsta_idx,dt_sta,dur_sta, no_neighbpix_x,locc, locc_, minc, locr, locr_, minr, popt_on,mappedrf_on,\
              minsta_idx, popt_off, mappedrf_off):
    fntsz = 12
    darkyellow = [255./255,200./255,0./255]# [238./255,238./255,0]
    x = np.linspace(0, no_neighbpix_x-1, no_neighbpix_x)
    y = np.linspace(0, no_neighbpix_x-1, no_neighbpix_x)
    x, y = np.meshgrid(x, y) 
    plt.figure(figsize=(8,6))   
    plt.subplot(221)
    plt.plot(sta_ich[:,neighborpixels_pos], color='grey')
    plt.hold('on')
    #  plt.plot(sta_ich[:,maxsta_idx[1]], color='r')

    #  plt.title('STA - '+ 'spkcnt ' + str(spkcnt_ich) , fontsize=10)
    plt.xlabel('Time (s)', fontsize=fntsz)
    plt.ylabel('Pixel brightness', fontsize=fntsz)
    plt.xticks(np.arange(0, len(sta_ich[:,0]), 1/dt_sta_graphics), time_bins)
    plt.ylim([0,1])
    polish_fig(plt.gca(),xlabcoord=-.15,vis_rightaxis=True, vis_leftaxis=False, vis_topaxis=False, vis_bottomaxis=True, xtick_pos='bottom', ytick_pos='right',xlabel_pos='bottom', ylabel_pos='right',boldvline=False)
    plt.subplots_adjust(hspace=.4)

    # Plotting firing rate*******
    #TODO  
    plt.subplot(222)
    plt.axis('off')

    plt.subplot(223)
    img_on = sta_ich[maxsta_idx[0],neighborpixels_pos]   
    #  sta_f = scipy.io.loadmat('/Users/sahar/Documents/spktanlz/data/retina/P38_06Mar14/ret2/Luiz/neuron_789_maxsta_SA2.mat', squeeze_me=True)
    #  img_on = np.hstack(sta_f['mean_sta'].transpose().reshape([27*27,1]))
    plt.title('Positive peak (t = '+ str(int((maxsta_idx[0]*dt_sta - dur_sta)*1000)) + ' ms)' , fontsize=10, color='k')
    plt.imshow(np.reshape(img_on, [no_neighbpix_x, no_neighbpix_x]), cmap=pylab.cm.gray, interpolation='none')
    plt.text(locc-minc-.5, locr-minr-.5,'+',color='r',horizontalalignment='center',verticalalignment='center', fontsize=fntsz)

    if popt_on.any():  
        data_fitted_on = Gaussian2D_bivariate((x, y), *popt_on) 
        if data_fitted_on.any():
          plt.contour(x, y, data_fitted_on.reshape(no_neighbpix_x, no_neighbpix_x), [Gaussian2D_bivariate((popt_on[0]+popt_on[3], popt_on[1]+popt_on[4]), *popt_on)], colors='k', linewidths=2)
          plt.text(popt_on[0],popt_on[1],'+',color='k',horizontalalignment='center',verticalalignment='center', fontsize=fntsz)
          if mappedrf_on:
            plt.title('Positive peak (t = '+ str(int((maxsta_idx[0]*dt_sta - dur_sta)*1000)) + ' ms)' , fontsize=fntsz, color=darkyellow)
            plt.contour(x, y, data_fitted_on.reshape(no_neighbpix_x, no_neighbpix_x), [Gaussian2D_bivariate((popt_on[0]+popt_on[3], popt_on[1]+popt_on[4]), *popt_on)], colors='orange', linewidths=2)
            plt.text(popt_on[0],popt_on[1],'+',color='orange',horizontalalignment='center',verticalalignment='center', fontsize=fntsz)
    plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    plt.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')

    plt.subplot(224)
    img_off = sta_ich[minsta_idx[0],neighborpixels_pos]   
    plt.title('Negative peak (t = '+ str(int((minsta_idx[0]*dt_sta - dur_sta)*1000)) + ' ms)' , fontsize=fntsz)
    plt.imshow(np.reshape(img_off, [no_neighbpix_x, no_neighbpix_x]), cmap=pylab.cm.gray, interpolation='none')
    plt.text(locc-minc-.5, locr-minr-.5,'+',color='r',horizontalalignment='center',verticalalignment='center', fontsize=fntsz)
    if popt_off.any():
        data_fitted_off = Gaussian2D_bivariate((x, y), *popt_off)
        if data_fitted_off.any():
          plt.contour(x, y, data_fitted_off.reshape(no_neighbpix_x, no_neighbpix_x), [Gaussian2D_bivariate((popt_off[0]+popt_off[3], popt_off[1]+popt_off[4]), *popt_off)], colors='k', linewidths=2)
          plt.text(popt_off[0], popt_off[1],'+',color='k',horizontalalignment='center',verticalalignment='center', fontsize=fntsz)
        #      pdb.set_trace()
          if mappedrf_off:
            plt.title('Negative peak (t = '+ str(int((minsta_idx[0]*dt_sta - dur_sta)*1000)) + ' ms)' , color=tableau[20],fontsize=fntsz)
            plt.contour(x, y, data_fitted_off.reshape(no_neighbpix_x, no_neighbpix_x), [Gaussian2D_bivariate((popt_off[0]+popt_off[3], popt_off[1]+popt_off[4]), *popt_off)], colors='b', linewidths=2)
            plt.text(popt_off[0], popt_off[1],'+',color=tableau[20],horizontalalignment='center',verticalalignment='center', fontsize=fntsz)
    plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off')
    plt.tick_params(axis='y', which='both', right='off', left='off', labelleft='off')

    plt.suptitle(spksrc+ str(ich)+ ' - loc (' + "%0.1f" % locr_ + ',' + "%0.1f" % locc_ + ')', fontsize=fntsz, y=.99)    
    figname = os.path.join(figpath,'sta_'+spksrc+str(ich)+'_spkcnt'+str(spkcnt_ich)+'dur'+str(dur_sta)+'dt'+str(dt_sta)+figformat)
    plt.savefig(figname, bbox_inches='tight')  
    print 'figure saved to:', figname    
    plt.close()

polish_fig(ax) polishes the figure passed by its axis, ax

In [166]:
def polish_fig(ax,xlabcoord=-.12,vis_rightaxis=False, vis_leftaxis=True, vis_topaxis=False, vis_bottomaxis=True, xtick_pos='bottom', ytick_pos='left', xlabel_pos='bottom', ylabel_pos='left', boldvline=True, invis_firstlabel=False, a=2):
  mplt.rcParams['xtick.direction'] = 'out'
  mplt.rcParams['ytick.direction'] = 'out'
  mplt.rc('axes',facecolor='ffffff')
  mplt.rc('axes',edgecolor='000000')
  mplt.rc('axes',labelcolor='000000')
  mplt.rc('xtick',color='000000')
  mplt.rc('ytick',color='000000')
  ax.spines["right"].set_visible(vis_rightaxis)
  ax.spines["top"].set_visible(vis_topaxis)
  ax.spines["left"].set_visible(vis_leftaxis)
  ax.spines["bottom"].set_visible(vis_bottomaxis)
  ax.xaxis.set_ticks_position(xtick_pos)
  ax.yaxis.set_ticks_position(ytick_pos)
  ax.xaxis.set_label_position(xlabel_pos)
  ax.yaxis.set_label_position(ylabel_pos)
  ax.xaxis.set_label_coords(0.5, xlabcoord)
      
  if invis_firstlabel:
    plt.setp(ax.get_yticklabels()[0], visible=False)    
    plt.setp(ax.get_xticklabels()[0], visible=False)    

#  ax.yaxis.set_major_locator(MaxNLocator(prune='lower'))
#  ax.tick_params(axis='x', pad=15) # distant ticks
  if boldvline:
    ax.axvline(linewidth=a, color="black")
    ax.axhline(linewidth=a, color="black")
    ax.tick_params('both', width=a, which='major', direction='out')

Gaussian2D_bivariate((x, y)) fits a gaussian2d 

In [167]:
def Gaussian2D_bivariate((x, y), xo, yo, amplitude, sigma_x, sigma_y, theta, offset):
    xo = float(xo)
    yo = float(yo)    
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo) 
                            + c*((y-yo)**2)))
    return g.ravel()

## Computing Spike-triggered Average

In [168]:
# assigning data variables
ch = data['ch']
no_ch = len(np.unique(ch))  
freq = data['freq']
spikes = data['spikes']
loc = data['loc']
loc2d = data['loc2d']
nbins = data['nbins']

# assigning stimulus variables
stimframes = stim['stimframes']
stimpixels = stim['stimpixels']
image_xsize = int(np.sqrt(len(stimpixels[0])))
dur_sta_f = int(dur_sta * freq)
spklatency_sta_f = int(spklatency_sta * freq)
# change in v1.2
dur_stim_est = int(np.average(stimframes[1:-1]-stimframes[0:-2]))
print '\nEstimated stimulus duration:', dur_stim_est/float(freq)*1000, 'ms, i.e. ', dur_stim_est,' frames\n'
# change end


Estimated stimulus duration: 40.7816482583 ms, i.e.  288  frames



Removing any pauses in the white-noise experiment: We shift spikes after experiment breaks as if there were no breaks in the experiemnt (a 15min experiment is usually devided to 3 5min experiment with pauses in between)

In [169]:
stimframes_idxpause = [idx for idx, elm in enumerate(stimframes[:-1]) if (stimframes[idx+1] - elm)>dur_stim_est*2]
print 'frames at which breaks happens (starting from 0):', stimframes_idxpause
spkidx = np.array([], dtype=int)
startf_idx = 0
no_neighbpix_x = np.sqrt(no_neighbpix)
#change in v1.2 (introducing avg_stimdur to estimate the real duration between stimuli after removing pauses)
avg_stimdur = []
for i, idx_p in enumerate(stimframes_idxpause):
    avg_stimdur.append(np.average(stimframes[startf_idx+1:idx_p]-stimframes[startf_idx:idx_p-1]))
    print 'stim pauses:', i, idx_p
    spkidx_spcf = get_spkidx(data, minamp, stimframes[startf_idx]+dur_sta_f+spklatency_sta_f, stimframes[idx_p]+spklatency_sta_f)
    spkidx = np.append(spkidx, spkidx_spcf)
    startf_idx = idx_p+1
# we faced a boundary problem, so in the following line we had to change stimframes[-1] to stimframes[-2], not a big deal really.
spkidx_spcf = get_spkidx(data, minamp, stimframes[startf_idx]+dur_sta_f+spklatency_sta_f, stimframes[-2]+spklatency_sta_f)
spkidx = np.append(spkidx, spkidx_spcf)

dur_stim_f = np.average(avg_stimdur)
dur_stim = dur_stim_f / float(freq)
print '\nThe true duration of stimulus:', dur_stim*1000, 'ms, i.e.', dur_stim_f, 'frames'
no_bins_sta = int(round(dur_sta / dt_sta))
no_bins_sta_stim = int(round(dur_sta / dur_stim))
# change end

frames at which breaks happens (starting from 0): [2999, 5999, 8999, 11999, 14999, 17999, 20999, 23999]
stim pauses: 0 2999
stim pauses: 1 5999
stim pauses: 2 8999
stim pauses: 3 11999
stim pauses: 4 14999
stim pauses: 5 17999
stim pauses: 6 20999
stim pauses: 7 23999

The true duration of stimulus: 33.3256308983 ms, i.e. 235.345605404 frames


Selecting spiking channels whose STA will be computed

In [170]:
hist_spkcnt = get_hist_spkcnt(data, spkidx)
allspikingChannels = np.where(hist_spkcnt >= minspk)[0] # if all channels spike: ~= np.arange(no_ch)
if slctdchs_sta == 'all':
    spikingChannels = allspikingChannels
else:
    spikingChannels = slctdchs_sta[np.where(hist_spkcnt[slctdchs_sta]>=minspk)[0]]

print 'spiking channels:', spikingChannels

# preparing the binning of the sta array
no_bins_sta = int(dur_sta / dt_sta)
no_bins_sta_stim = int(dur_sta / dur_stim)
if no_bins_sta_stim > no_bins_sta:
    raise ValueError('You should decrease dt_sta (i.e. increasing the resolution) for this stimulus')
no_bins_add = no_bins_sta / no_bins_sta_stim
stimidx_rep = np.repeat(range(len(stimframes)), no_bins_add)
time_bins = np.arange(-dur_sta*10, 1, dt_sta_graphics*10)/10.

spiking channels: [ 275 1071]


Allocating memories 
(TODO: it needs to be optimized)

In [171]:
stalatency_on, stalatency_off = (np.zeros(max(spikingChannels)+1), np.zeros(max(spikingChannels)+1))
stapeakresp_on, stapeakresp_off = (np.ones(max(spikingChannels)+1)*.5, np.ones(max(spikingChannels)+1)*.5)
rfdiameters_on, rfdiameters_off = (np.zeros(max(spikingChannels)+1), np.zeros(max(spikingChannels)+1))
starespunits, mappedrfunits = (np.zeros(max(spikingChannels)+1), np.zeros(max(spikingChannels)+1))
popts_on = np.zeros([max(spikingChannels)+1, no_gausfit_params])
popts_off = np.zeros([max(spikingChannels)+1, no_gausfit_params])
locs_spikingChs = np.zeros((max(spikingChannels)+1,2))

# params returned after computation of firing rate  
ffspkcnt2d_avg, ffspkcnt2d_std, ffspkcnt2l_avg, ffspkcnt2l_std = (np.zeros(max(spikingChannels)+1) for i in range(4))
fflatency2dark, fflatency2light, ffduresp2dark, ffduresp2light, ffpeakresp2dark, ffpeakresp2light = (np.zeros(max(spikingChannels)+1) for i in range(6))
ffirstspk2d_avg, ffirstspk2d_std, ffirstspk2l_avg, ffirstspk2l_std = (np.zeros(max(spikingChannels)+1) for i in range(4))
biasidxs = np.ones(max(spikingChannels)+1) * -1.2

Computing STA for each channel

In [172]:
for ich in spikingChannels:
    print '\n', spksrc, ':', str(ich), '*************'
    
    # Plot and Compute Bias Index if necessary ******************************
    if calcBiasidx4staFLG:
        pass
        # TODO: its ipython notebook needs to be implemented

    popt_on, popt_off = (np.array([]),np.array([]))
    pcov_on, pcov_off = (np.array([]),np.array([]))
    locr_, locc_ = get_chlocation(ich, spksrc, nbins, loc, coord_spksrc)
    if spksrc == 'cl':
      locr_, locc_ = get_chlocation(ich, spksrc, nbins, loc2d, coord_spksrc) 
    else:
      locr_, locc_ = get_chlocation(ich, spksrc, nbins, loc, coord_spksrc) 
    locs_spikingChs[ich] = np.array([locr_, locc_])

    locr = (locr_ + .5) * image_xsize/64.0 
    locc = (locc_ + .5) * image_xsize/64.0 
    neighborpixels_pos, minr, minc = get_neighborpixels(int(round(locr)), int(round(locc)), image_xsize, no_neighbpix_x)
    spkidx_ich = np.where(ch==ich)[0]
    spkidx_ich = np.intersect1d(spkidx_ich, spkidx)
    sta_ich = np.zeros([no_bins_sta+1, len(stimpixels[0])])

    # estimating the start frames of STA; for example, if the first spike of ich is fired at frame 30000, i.e. spikes[spkidx_ich][0]==30000, and dur_sta_f=500ms*7022, then the first sta frame is startfs_sta_ich[0]=30000-.5*7022
    startfs_sta_ich = spikes[spkidx_ich] - dur_sta_f - spklatency_sta_f
    
    #joins the paused frames together (removing pauses within white noise presentation and aligning frames for computing STA)
    startfs_sta_ich_tmp = np.copy(startfs_sta_ich)   
    for pausef_idx in stimframes_idxpause:
      idx_afterp = np.where(startfs_sta_ich > stimframes[pausef_idx])
      startfs_sta_ich_tmp[idx_afterp] -= stimframes[pausef_idx+1] - stimframes[pausef_idx] - dur_stim_f
    startfs_sta_ich_idx = [int(x) for x in (startfs_sta_ich_tmp-stimframes[0])/float(dur_stim_f)*no_bins_add]

    # computing STA of ich: averaging stimulus pixels across a particular time before spikes
    for idx, startfi in enumerate(startfs_sta_ich_idx): 
      #print '\n***\nspike frame:', spikes[spkidx_ich[idx]], '\nstartframe:', startfs_sta_ich_tmp[idx], '\n sta frames:', [stimframes[0]+(i*dur_stim_f/no_bins_add) for i in range(startfi,startfi+no_bins_sta+1)], '\n stimpixels:', stimpixels[stimidx_rep[startfi:startfi+no_bins_sta+1],0],'\n'
      sta_ich += stimpixels[stimidx_rep[startfi:startfi+no_bins_sta+1]]
    sta_ich /= len(spkidx_ich)

    #*************************************************
    # finding positive and negative peak STAs 
    maxsta_idx = np.unravel_index(sta_ich.argmax(), sta_ich.shape)
    minsta_idx = np.unravel_index(sta_ich.argmin(), sta_ich.shape)

    # Calculating the zscores of positive and negative peak STAs: (x-mu)/var  
    zscores_atmaxsta = stats.zscore(sta_ich[maxsta_idx[0],neighborpixels_pos])
    zscores_atminsta = stats.zscore(sta_ich[minsta_idx[0],neighborpixels_pos])
    
    # calculating p-values of peak STAs by computing the cumulative density function at peaks
    # scipy.special.ndtr returns cumulative density function, i.e. the area under the probability density function of the given zscores
    pval_atmaxsta = scipy.special.ndtr(-np.abs(zscores_atmaxsta[maxsta_idx[1]]))
    pval_atminsta = scipy.special.ndtr(-np.abs(zscores_atminsta[minsta_idx[1]]))
    
    if (checkSTAPeakZFLG and pval_atmaxsta > max_pval_acceptable and pval_atminsta > max_pval_acceptable):
      continue
    #    pdb.set_trace()

    # Evaluate ON response ***************************************************
    mappedrf_on = 0
    x = np.linspace(0, no_neighbpix_x-1, no_neighbpix_x)
    y = np.linspace(0, no_neighbpix_x-1, no_neighbpix_x)
    x, y = np.meshgrid(x, y) 
    img_on = sta_ich[maxsta_idx[0],neighborpixels_pos]   
    # reading sta directly from a file****************
    #    pdb.set_trace()
    #    sta_f = scipy.io.loadmat('/Users/sahar/Documents/spktanlz/data/retina/P38_06Mar14/ret2/Luiz/neuron_789_maxsta_SA2.mat', squeeze_me=True)
    #    img_on = np.hstack(sta_f['mean_sta'].transpose().reshape([27*27,1]))

    initial_guess = [locc-minc-.5, locr-minr-.5, 1, 1, 1, 0, 0]
    #    initial_guess = [locc-minc-1, locr-minr-1, 1, 1, 1, 0, 0]
    try:
      popt_on, pcov_on = opt.curve_fit(Gaussian2D_bivariate, (x,y), img_on, p0=initial_guess)
      popts_on[ich] = popt_on
      popt_onstr = [float("%0.2f" % v) for v in popt_on]
      popt_var_on = [float("%0.2f" % v) for v in pcov_on.diagonal()]
      if PrintGaussParamFLG:
        print '\nON: Gaussian2D was fit by the parameters:\nxo=', popt_onstr[0], '\nyo=', popt_onstr[1], \
              '\namplitude=', popt_onstr[2] , '\nsigma_x=', popt_onstr[3], '\nsigma_y=', popt_onstr[4],'\ntheta=', popt_onstr[5], '\noffset=', popt_onstr[6]     
        print '\nON: Variance of the parameters:\nxo_var=', popt_var_on[0], '\nyo_var=', popt_var_on[1], \
              '\namplitude_var=', popt_var_on[2] , '\nsigma_x_var=', popt_var_on[3], '\nsigma_y_var=', popt_var_on[4],'\ntheta_var=', popt_var_on[5], '\noffset_var=', popt_var_on[6]     

    #      data_fitted_on = Gaussian2D_bivariate((x, y), *popt_on)      
      #pval_atmaxsta < max_pval_acceptable and             
      if (np.abs(popt_var_on[0]) < max_var_acceptable and np.abs(popt_var_on[1]) < max_var_acceptable and popt_onstr[2] > 0):
        print '***On-response RF was mapped ***'
        mappedrf_on = 1
    except:
      print '\n!!!on-response could not be mapped\n'

    # Evaluate OFF response ***************************************************
    mappedrf_off = 0
    img_off = sta_ich[minsta_idx[0],neighborpixels_pos]   
    initial_guess = [locc-minc-.5, locr-minr-.5, 1, 1, 1, 0, 0]

    try:
      popt_off, pcov_off = opt.curve_fit(Gaussian2D_bivariate, (x, y), img_off, p0=initial_guess)
      popts_off[ich] = popt_off
      popt_offstr = [float("%0.2f" % v) for v in popt_off]
      popt_var_off = [float("%0.2f" % v) for v in pcov_off.diagonal()]
      if PrintGaussParamFLG:
        print '\n***\nOFF: Gaussian2D was fit by the parameters:\nxo=', popt_offstr[0], '\nyo=', popt_offstr[1], \
              '\namplitude=', popt_offstr[2] , '\nsigma_x=', popt_offstr[3], '\nsigma_y=', popt_offstr[4],'\ntheta=', popt_offstr[5], '\noffset=', popt_offstr[6] #Gaussian2D_bivariate
        print '\nOFF: Variance of the parameters:\nxo_var=', popt_var_off[0], '\nyo_var=', popt_var_off[1], \
              '\namplitude_var=', popt_var_off[2] , '\nsigma_x_var=', popt_var_off[3], '\nsigma_y_var=', popt_var_off[4],'\ntheta_var=', popt_var_off[5], '\noffset_var=', popt_var_off[6]     
    #      data_fitted_off = Gaussian2D_bivariate((x, y), *popt_off)
      # pval_atminsta < max_pval_acceptable
    #      pdb.set_trace()
      if (np.abs(popt_var_off[0]) < max_var_acceptable and np.abs(popt_var_off[1]) < max_var_acceptable and popt_offstr[2] < 0):
        print '***OFF-response RF was mapped ***'
        mappedrf_off = 1
    except:
      print '\n!!!off-response could not be mapped\n'

    # Setting STA parameters **************************************
    isvalid_tempstaon = (pval_atmaxsta < max_pval_acceptable)
    isvalid_tempstaoff = (pval_atminsta < max_pval_acceptable)
    if isvalid_tempstaon:
      stalatency_on[ich] = (dur_sta - maxsta_idx[0]*dt_sta)*1000
      stapeakresp_on[ich] = sta_ich.max()
    if isvalid_tempstaoff:
      stalatency_off[ich] = (dur_sta - minsta_idx[0]*dt_sta)*1000
      stapeakresp_off[ich] = sta_ich.min()

    #    pdb.set_trace()
    if (isvalid_tempstaon and isvalid_tempstaoff):    
      # if ON peak is nearer to spike than OFF peak
      if maxsta_idx[0] > minsta_idx[0]:
        starespunits[ich] = 1
      else:
        starespunits[ich] = -1
    elif isvalid_tempstaon:
      starespunits[ich] = 1      
    elif isvalid_tempstaoff:
      starespunits[ich] = -1

    # Setting the RF parameters **************************************
    #    pdb.set_trace()
    if mappedrf_on:
      rfdiameters_on[ich] = 2 * np.sqrt(np.abs(popt_on[3] * popt_on[4]))
    if mappedrf_off:
      rfdiameters_off[ich] = 2 * np.sqrt(np.abs(popt_off[3] * popt_off[4]))

    if mappedrf_on and mappedrf_off:    
      # if ON peak is nearer to spike than OFF peak
      if (starespunits[ich] == 1):
        mappedrfunits[ich] = 1
      else:
        mappedrfunits[ich] = -1
    elif mappedrf_on:
      mappedrfunits[ich] = 1
    elif mappedrf_off:
      mappedrfunits[ich] = -1
    
    # receptive field diameters
    rfdiameters_on[ich] *= chip_size * 1000 * 1./image_xsize     
    rfdiameters_off[ich] *= chip_size * 1000 * 1./image_xsize     

    stapeakresp_on[ich] = np.abs(0.5 - stapeakresp_on[ich])
    stapeakresp_off[ich] = np.abs(0.5 - stapeakresp_off[ich])
    #    pdb.set_trace()     
    # Plot STA ***************************************************************
    if staplotFLG and (mappedrf_on or mappedrf_off or not plotJustRespUnitsFLG): 
      plotsta(ich, sta_ich, neighborpixels_pos, hist_spkcnt[ich], dt_sta_graphics, time_bins,figpath, spksrc,\
              figformat,maxsta_idx,dt_sta,dur_sta, no_neighbpix_x,locc, locc_, minc, locr, locr_, minr, popt_on,mappedrf_on,\
              minsta_idx, popt_off, mappedrf_off)
        
if staSaveParamsFLG:
    print 'saving sta params in a mat file...'
    paramnames = np.array(['Spikingchannels', 'locs_r', 'locs_c', 'biasidxs', 'latency2l', 'latency2d',\
              'duresp2l', 'duresp2d', 'peakresp2l', 'peakresp2d', 'firstspk2l_avg', 'firstspk2d_avg', \
              'firstspk2l_std', 'firstspk2d_std', 'spkcnt2l_avg', 'spkcnt2d_avg', 'spkcnt2l_std', 'spkcnt2d_std', \
              'starespunits', 'stalatency_on', 'stalatency_off', 'stapeakresp_on', 'stapeakresp_off', 'mappedrfunits', 'rfdiameters_on', 'rfdiameters_off', \
              'rf_on_x0', 'rf_on_y0', 'rf_on_amp', 'rf_on_sigmax', 'rf_on_sigmay', 'rf_on_theta', 'rf_on_offset',\
              'rf_off_x0', 'rf_off_y0', 'rf_off_amp', 'rf_off_sigmax', 'rf_off_sigmay', 'rf_off_theta', 'rf_off_offset'])
    params = np.transpose(np.vstack([spikingChannels, locs_spikingChs[spikingChannels, 0], locs_spikingChs[spikingChannels, 1], biasidxs[spikingChannels], fflatency2light[spikingChannels], fflatency2dark[spikingChannels],\
              ffduresp2light[spikingChannels], ffduresp2dark[spikingChannels], ffpeakresp2light[spikingChannels], ffpeakresp2dark[spikingChannels], ffirstspk2l_avg[spikingChannels], ffirstspk2d_avg[spikingChannels], \
              ffirstspk2l_std[spikingChannels], ffirstspk2d_std[spikingChannels], ffspkcnt2l_avg[spikingChannels], ffspkcnt2d_avg[spikingChannels], ffspkcnt2l_std[spikingChannels], ffspkcnt2d_std[spikingChannels],\
              starespunits[spikingChannels], stalatency_on[spikingChannels], stalatency_off[spikingChannels], stapeakresp_on[spikingChannels], stapeakresp_off[spikingChannels], mappedrfunits[spikingChannels], rfdiameters_on[spikingChannels], rfdiameters_off[spikingChannels], \
              popts_on[spikingChannels,0], popts_on[spikingChannels,1],popts_on[spikingChannels,2],popts_on[spikingChannels,3],popts_on[spikingChannels,4],popts_on[spikingChannels,5],popts_on[spikingChannels,6],\
              popts_off[spikingChannels,0], popts_off[spikingChannels,1],popts_off[spikingChannels,2],popts_off[spikingChannels,3],popts_off[spikingChannels,4],popts_off[spikingChannels,5],popts_off[spikingChannels,6]]))

    params = params.astype(object)
    params = np.insert(params, 0, paramnames, axis=0)
#    pdb.set_trace()
    paramsf = staparams_matfile+'_'+str(len(spikingChannels))+spksrc+'.mat'
    scipy.io.savemat(paramsf,mdict={'STAfrParams':params}, oned_as='column')
    print 'Parameters were saved into: ', paramsf    


su : 275 *************
***On-response RF was mapped ***
***OFF-response RF was mapped ***
figure saved to: /Users/sahar/Documents/spktanlz/data/retina/P79_28Oct14/figs/sta_su275_spkcnt2441dur0.5dt0.01.png

su : 1071 *************
saving sta params in a mat file...
Parameters were saved into:  /Users/sahar/Documents/spktanlz/data/retina/P79_28Oct14/figs/staparams_2su.mat
